## K-Nearest Neighbours (KNN)

Pasos para aplicar KNN
1. Preparación de datos
1. Ajuste de KNN al conjunto de entrenamiento
1. Predecir los resultados de las pruebas
1. Comprobar la precisión de los resultados
1. Realizar predicción

## Paso 1: carga y preparación de datos

- Importación

In [27]:
# Importar librearias necesarias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

- Carga de datos

In [28]:
# Importar el dataset
dataset = pd.read_csv('user+data.csv')
dataset.head(4)


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0


- Exploración de datos

In [29]:
print(f"Mis datos tienen : {dataset.shape[1]:,} columnas")
print(f"Mis datos tienen : {dataset.shape[0]:,} filas")
print(f"Para un total de {dataset.shape[0]*dataset.shape[1]:,} datos")

Mis datos tienen : 5 columnas
Mis datos tienen : 400 filas
Para un total de 2,000 datos


In [30]:
# Identificamos cuantos son NULL para un posible tratamiento

dataset.isnull().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

- Integridad de los datos

In [31]:
cantidad = dataset.count() # Cantidad de datos
cant_nulos = dataset.isnull().sum() # Cantidad de registros nulos
acum_nulos = cant_nulos.cumsum() # Sumar los nulos totales fila a fila
porc_nulos = (dataset.isnull().sum()/dataset.shape[0])*100 # Porcentaje de nulos por campo


# Construir dataFrame con valores
resumen_tabla = pd.DataFrame({
    'Tipo' : dataset.dtypes,
    'Cantidad': cantidad.astype(int), # Convertir en numeros Enteros
    'Cant nulos':cant_nulos,
    'Acumu nulos': acum_nulos,
    'Porc nulos (%)': porc_nulos.round(2) # Redondear a 2 unidades decimales
})


resumen_tabla.reset_index(inplace= True) # Convertir el indice en una columa
 
resumen_tabla.rename(columns = {'index':'Columnas'}, inplace= True) # Cambiar el nombre de esa nueva columna


resumen_tabla

,Columnas,Tipo,Cantidad,Cant nulos,Acumu nulos,Porc nulos (%)
0,User ID,int64,400,0,0,0.0
1,Gender,object,400,0,0,0.0
2,Age,int64,400,0,0,0.0
3,EstimatedSalary,int64,400,0,0,0.0
4,Purchased,int64,400,0,0,0.0


- Resumen de datos

In [32]:
dataset[['Age', 'EstimatedSalary' ,'Purchased']].describe().round(2)

,Age,EstimatedSalary,Purchased
count,400.00,400.00,400.00
mean,37.66,69742.50,0.36
std,10.48,34096.96,0.48
min,18.00,15000.00,0.00
25%,29.75,43000.00,0.00
50%,37.00,70000.00,0.00
75%,46.00,88000.00,1.00
max,60.00,150000.00,1.00


## Paso 2: Ajuste de KNN al conjunto de entrenamiento

- Seleccionar datos

    El objetivo es identificar cual será nuestra(s) variable(s) independiente(s) y cual será nuestra variable objetivo. Recordamos que solo el aprendizaje supervisado tiene una variable objetivo o dependiente.

In [33]:
X = dataset.iloc[:, 2:4].values  # Nuestra matriz "X" en mayusucula
y = dataset.iloc[:, 4].values # Nuestro vector "y" en minuscula

- Dividir el conjunto de datos 

    En este paso queremos separar un 75% para datos de entrenamiento y 25% para datos de pruebas. Recordemos que ese 25% de pruebas son registros a los cuales el modelo no tendrá acceso, sino que despues de entrenar el modelo usaremos para comparar como es la precisión de la predicción

In [34]:
from sklearn.model_selection import train_test_split

# el random_state es la semilla aleatoria util para poder replicar los resultados

X_train, X_test, y_train, y_test =train_test_split(X,y,test_size= 0.25, random_state = 0)

- Verificar las salidas de X_train, X_test, y_train, y_test

    A continuación, podemos verificar que el 75% de los datos son 300 filas, de un total de 400 filas. Lo convertimos en una DataFrame para que veamos con nombres de columnas. Sin embargo, "X_train" recordamos que es una matriz, por lo tento no tienen nombres de columnas.

    Este paso solo es para que comprendan la salida de cada uno

In [35]:
pd.DataFrame(X_train, columns= ['Edad','Salario'])

,Edad,Salario
0,44,39000
1,32,120000
2,38,50000
3,32,135000
4,52,21000
...,...,...
295,48,30000
296,29,43000
297,36,52000
298,27,54000


- Estandarizar los datos para evitar que el modelo sobreestime alrededor de los valores o magnitudes mas altos. Muchos algoritmos (como KNN) comparan distancias entre puntos. Si una variable (como el salario) tiene números muy grandes comparado con otra (como la edad), el modelo se deja influenciar mucho más por el salario.

- fit() → El escalador aprende cómo están tus datos de entrenamiento: Calcula la media (promedio) y la desviación estándar de cada columna.

- transform() → Luego transforma todos los valores usando la fórmula:

    $$ \text{nuevo valor} = \frac{\text{valor original} - \text{media}}{\text{desviación estándar}} $$


    Esto hace que los datos queden centrados en 0 y tengan una desviación estándar de 1.




In [36]:
from sklearn.preprocessing import StandardScaler

# Creamos la instancia donde guardaremos el modelo
sc_X = StandardScaler()

# El fit calcula media y el transform aplica media, varianza = 0
X_train = sc_X.fit_transform(X_train)

# Usamos la estandarizacion aprendida
X_test = sc_X.transform(X_test)

### Paso 2.1: Entrenamiento con el clasificador KNN

- `n_neighbors = 5`
    Esto indica que el modelo usará los 5 vecinos más cercanos para decidir la clase de un nuevo punto.

- `metric = "minkowski" `
    minkowski es una métrica generalizada (por defecto) para calcular distancias.

- ` p=2: `
    Es un parámetro por defecto que se usa para calcular la distancia Euclidiana.

In [37]:
from sklearn.neighbors import KNeighborsClassifier


classifier = KNeighborsClassifier(n_neighbors=5, metric="minkowski", p=2)

# Entrenamos el modelo
classifier.fit(X_train,y_train)

KNeighborsClassifier()

## Paso 3: Predecir con los valores de prueba

In [38]:
from sklearn.metrics import confusion_matrix

# Calculamos las predicciones con los valores de pruebas
y_pred = classifer.predict(X_test)

A continuación, comparamos las predicciones realizadas con los valores de prueba ` X_test` contra las **salidas reales** de nuestro conjunto de prueba

In [39]:

df = pd.DataFrame({'Real Values':y_test, 'Predicted Values':y_pred})
df.head(5)

,Real Values,Predicted Values
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


Queremos realizar un conteo de cuantos predicciones fueron correctas y cuantas incorrectas

In [40]:
# Añadir columna de comparación
df['Match'] = df['Real Values'] == df['Predicted Values']

# Contar coincidencias y diferencias
conteo = df['Match'].value_counts()

print("Conteo de coincidencias:\n")
print(conteo)

Conteo de coincidencias:

Match
True     93
False     7
Name: count, dtype: int64


## Paso 4: Comprobar la precisión

Usamos la Matriz de Confusión

In [41]:
cm = confusion_matrix(y_pred, y_test)
print(cm)

[[64  3]
 [ 4 29]]


De los 7 valores que no habian coincidido (False = 7) entonces:

- 3 valores falsos se predicen incorrectamente verdaderos
- 4 valores verdaderos se predicen incorrectamente falsos

En términos generales:

- 64 valores verdaderos se predicen correctamente verdaderos
- 3 valores falsos se predicen incorrectamente verdaderos
- 4 valores verdaderos se predicen incorrectamente falsos
- 29 valores falsos se predicen correctamente falsos

### Evaluación del modelo

In [42]:
from sklearn.metrics import accuracy_score

# Comparar las predicciones con los valores reales
accuracy = accuracy_score(y_test, y_pred)

# Mostrar como porcentaje
print(f"Precisión del modelo: {accuracy * 100:.2f}%")

Precisión del modelo: 93.00%


## Paso 5: Realizar predicciones con el modelo entrenado

In [43]:
# Pedir datos al usuario
age = 25  # float(input("Ingresa la edad: "))
salary =  25000  # float(input("Ingresa el salario mensual: "))

# Crear array con los datos ingresados
input_data = [[age, salary]]

# Escalamos los valores reales antes de predecir
input_data_scaled = sc_X.transform(input_data)
predicted_class = classifier.predict(input_data_scaled)


print(f"Edad ingresada: {age}")
print(f"Salario ingresado: {salary}")
print(f"Precisión del modelo: {accuracy * 100:.2f}%")
print(f"La clase predicha para Purchased (Comprado) es: {predicted_class[0]}")

Edad ingresada: 25
Salario ingresado: 25000
Precisión del modelo: 93.00%
La clase predicha para Purchased (Comprado) es: 0
